## Deliverable 2. Create a Customer Travel Destinations Map.

In [30]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Current Description
0,0,Muros,ES,42.7762,-9.0603,41.0,82,96,overcast clouds
1,1,Hasaki,JP,35.7333,140.8333,66.0,29,75,broken clouds
2,2,Hilo,US,19.7297,-155.0900,73.4,73,90,overcast clouds
3,3,Hirara,JP,24.8000,125.2833,71.6,53,40,scattered clouds
4,4,Olenegorsk,RU,68.1500,33.3000,36.1,89,81,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
# # Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pref_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
pref_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Current Description
2,2,Hilo,US,19.7297,-155.0900,73.40,73,90,overcast clouds
3,3,Hirara,JP,24.8000,125.2833,71.60,53,40,scattered clouds
5,5,Yulara,AU,-25.2406,130.9889,75.20,20,0,clear sky
6,6,Banda Aceh,ID,5.5577,95.3222,85.75,67,43,scattered clouds
7,7,Samarai,PG,-10.6167,150.6667,83.98,74,72,broken clouds
10,10,Teshie,GH,5.5836,-0.1072,78.80,78,0,clear sky
11,11,Boende,CD,-0.2167,20.8667,73.63,92,100,overcast clouds
12,12,Cape Town,ZA,-33.9258,18.4232,73.99,48,0,clear sky
13,13,Riyadh,SA,24.6877,46.7219,82.40,13,0,clear sky
22,22,Busselton,AU,-33.6500,115.3333,77.00,42,0,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
pref_cities_df.count()

City_ID                248
City                   248
Country                246
Lat                    248
Lng                    248
Max Temp               248
Humidity               248
Cloudiness             248
Current Description    248
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_clean_df = pref_cities_df.dropna()
pref_cities_clean_df.count()

City_ID                246
City                   246
Country                246
Lat                    246
Lng                    246
Max Temp               246
Humidity               246
Cloudiness             246
Current Description    246
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hilo,US,73.40,overcast clouds,19.7297,-155.0900,
3,Hirara,JP,71.60,scattered clouds,24.8000,125.2833,
5,Yulara,AU,75.20,clear sky,-25.2406,130.9889,
6,Banda Aceh,ID,85.75,scattered clouds,5.5577,95.3222,
7,Samarai,PG,83.98,broken clouds,-10.6167,150.6667,
10,Teshie,GH,78.80,clear sky,5.5836,-0.1072,
11,Boende,CD,73.63,overcast clouds,-0.2167,20.8667,
12,Cape Town,ZA,73.99,clear sky,-33.9258,18.4232,
13,Riyadh,SA,82.40,clear sky,24.6877,46.7219,
22,Busselton,AU,77.00,clear sky,-33.6500,115.3333,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found...skipping') 


Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [40]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = hotel_df.dropna()


In [44]:
# 8a. Create the output File (CSV)
hotel_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(hotel_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))